In [24]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
import sys
import keras
import keras.backend as K
sys.path.append("data")
from data.brainds import BrainTumorDataset
from models.unet import *
from callbacks import ImageHistory

In [8]:
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import keras

In [9]:
data_directory = './data/datasets/brain_tumor'
weight_file = './weights.hdf5'
tensor_board_root_directory = './logs'

In [19]:
dataset = BrainTumorDataset(data_directory)

In [4]:
dataset.download()

In [5]:
dataset.prepare()

In [11]:
learning_rate = 1e-4
input_shape=(512,512,1)
loss = 'binary_crossentropy'
steps_per_epoch = 1000
epochs = 3
batch_size = 5
start_epoch = 0
last_step = 10
clear_all = False

## Setup optimizer

In [14]:
optimizer = keras.optimizers.Adam(lr = learning_rate)

In [17]:
builder = Unet(input_shape=input_shape)
model = builder.build_model()
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
CNT_1_conv_1 (Conv2D)           (None, 512, 512, 64) 640         input[0][0]                      
__________________________________________________________________________________________________
CNT_1_conv_2 (Conv2D)           (None, 512, 512, 64) 36928       CNT_1_conv_1[0][0]               
__________________________________________________________________________________________________
CNT_2_maxpool (MaxPooling2D)    (None, 256, 256, 64) 0           CNT_1_conv_2[0][0]               
___________________________________________________________________________________________

## Dataset generators

In [20]:
train_generator = dataset.generator('training', batch_size=batch_size)
valid_generator = dataset.generator('validation', batch_size=1)

## Callbacks

In [22]:
tensor_board_directory = f'{tensor_board_root_directory}/{loss}-{learning_rate}'

In [26]:
# tensor board
tensorboard = keras.callbacks.TensorBoard(log_dir=tensor_board_directory, histogram_freq=0, write_graph=True, write_images=True, update_freq=100)
# image prediction history to be shown in the Tensor board
v_generator = dataset.generator('validation', batch_size=1)
v_data = [next(v_generator) for item in range(3)]
image_history = ImageHistory(tensor_board_dir=tensor_board_directory, data=v_data, last_step=last_step, draw_interval=100)
# model saving at end of an epoch
model_checkpoint = keras.callbacks.ModelCheckpoint(weight_file, monitor='loss',verbose=1, save_best_only=True)

In [27]:
def pixel_difference(y_true, y_pred):
    '''
    Custom metrics for comparison of images
    pixel by pixel. 
    '''
    cof = 100/(input_shape[0]*input_shape[1]*batch_size)
    return cof*K.sum(K.abs(y_true - y_pred))

metrics = ['accuracy', pixel_difference]

In [30]:
model.compile(optimizer=optimizer, loss = loss, metrics = metrics)

## Train Model

In [ ]:
model.fit_generator(generator=train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs, initial_epoch=start_epoch,
                    validation_data=v_generator, validation_steps=1,
                    callbacks=[model_checkpoint, tensorboard, image_history])




Epoch 1/3
   4/1000 [..............................] - ETA: 15:08:51 - loss: 0.6933 - accuracy: 0.7281 - pixel_difference: 50.0082